In [1]:
import pickle
import numpy as np
import pandas as pd

In [2]:
# relation file
relation_dict_path = 'data/relation_dict.pkl'
# train file
label_random_path = 'data/label_random.pkl'
label_gabor_path = 'data/label_gabor.pkl'
unlabel_data_file = 'data/DS_noise.pkl'
# eval file
group_eval_data_file = 'data/slim_test_group.pkl'

In [3]:
f=open(relation_dict_path, 'rb')
relation_dict = pickle.load(f)
f.close()

f=open(label_random_path, 'rb')
label_random = pickle.load(f)
f.close()

f=open(label_gabor_path, 'rb')
label_gabor = pickle.load(f)
f.close()

f=open(unlabel_data_file, 'rb')
unlabel_data = pickle.load(f)
f.close()

f=open(group_eval_data_file, 'rb')
group_eval_data = pickle.load(f)
f.close()

In [4]:
print(relation_dict)

def get_relation_counter():
    relation_counter = {}
    for r in list(relation_dict.keys()):
        relation_counter[r] = 0
    return relation_counter

{'per:origin': 1, '/people/person/place_of_birth': 2, 'NA': 0, '/people/person/place_lived': 3, '/people/deceased_person/place_of_death': 4}


In [5]:
sample = list(group_eval_data.keys())[0]
print(group_eval_data[sample][0])
print(group_eval_data[sample][1], len(group_eval_data[sample][1]))
print(group_eval_data[sample][2])

{'NA'}
[['with', 'the', 'end', 'of', 'the', 'northern', 'ireland', 'conflict', 'and', 'the', 'power-sharing', 'agreement', 'of', 'the', 'rev', '.', 'ian', 'paisley', 'of', 'the', 'democratic', 'unionist', 'party', 'and', 'martin', 'mcguinness', 'of', 'sinn', 'fein', ',', 'this', 'area', 'of', 'county', 'meath', 'has', 'rapidly', 'become', 'the', 'most', 'disputed', 'terrain', 'in', 'the', 'country', '.'], ['it', "'s", 'a', 'day', 'that', 'no', 'one', 'thought', 'ever', 'to', 'see', ',', 'ian', 'paisley', 'of', 'the', 'democratic', 'unionist', 'party', 'in', 'government', 'with', 'martin', 'mcguinness', 'of', 'sinn', 'fein', ',', '"', 'said', 'sydney', 'elliott', ',', 'a', 'professor', 'of', 'politics', 'at', 'queen', "'s", 'university', 'here', '.', '"'], ['ex-foes', 'sworn', 'in', 'to', 'lead', 'in', 'northern', 'ireland', 'from', 'left', ',', 'martin', 'mcguinness', ',', 'deputy', 'first', 'minister', 'of', 'the', 'northern', 'ireland', 'executive', 'government', ';', 'the', 'rev', '

In [6]:
import spacy
from spacy.tokens import Doc
nlp = spacy.load('en')
pos_vocab = {}
dep_vocab = {}

def get_pos_dep(tok_list):
    global pos_vocab
    global dep_vocab
    doc = Doc(nlp.vocab, words=tok_list)
    pos_list = []
    dep_list = []
    nlp.tagger(doc)
    nlp.parser(doc)
    for tok in doc:
        pos_list.append(tok.tag_)
        dep_list.append(tok.dep_)
        if tok.tag_ not in pos_vocab:
            pos_vocab[tok.tag_] = len(pos_vocab)
        if tok.dep_ not in dep_vocab:
            dep_vocab[tok.dep_] = len(dep_vocab)
    assert len(tok_list) == len(pos_list), "Tok list lenght doesn't match with pos length"
    assert len(tok_list) == len(dep_list)
    return pos_list, dep_list

def label_data_add_tag_dep(data):
    T, E, P, R = data
    POS = []
    DEP = []
    for tok_list in T:
        pos, dep = get_pos_dep(tok_list)
        POS.append(pos)
        DEP.append(dep)
    return (T, E, P, R, POS, DEP)

def group_data_add_tag_dep(group):
    data = group.copy()
    all_keys = list(data.keys())
    for key in all_keys:
        sentences_list = data[key][1]
        pos_list = []
        dep_list = []
        for sentence in sentences_list:
            pos, dep = get_pos_dep(sentence)
            pos_list.append(pos)
            dep_list.append(dep)
        data[key].append(pos_list)
        data[key].append(dep_list)
    return data

In [7]:
label_random_new = label_data_add_tag_dep(label_random)

In [8]:
label_gabor_new = label_data_add_tag_dep(label_gabor)

In [ ]:
unlabel_data_new = label_data_add_tag_dep(unlabel_data)

In [ ]:
group_eval_data_new = group_data_add_tag_dep(group_eval_data)

In [ ]:
def save_pickle(file_path, data):
    with open(file_path, 'wb'):
        pickle.dump(data)
    print('Done save to', file_path)

save_pickle('data/label_random_new.pkl', label_random_new)
save_pickle('data/label_gabor_new.pkl', label_gabor_new)
save_pickle('data/DS_noise_new.pkl', unlabel_data_new)
save_pickle('data/slim_test_group_new.pkl', group_eval_data_new)
save_pickle('data/pos_vocab.pkl', pos_vocab)
save_pickle('data/dep_vocab.pkl', dep_vocab)